# Exemplo

## Modelo cinético de reação de isomerização

Exemplo retirado de Schwaab e Pinto (2007, p.366) com base no exemplo encontrado em Schwaab e Pinto (2007, p.325) que trata sobre a estimação de parâmetros do modelo cinético:


![Exemplo de reação de isomerização](http://www.scielo.br/img/revistas/aa/v40n4/a14img01.jpg)


Função objetivo:

$y = \exp\left[-t\cdot \exp\left(K_0-\frac{E}{T}\right)\right]$,                  (1)

em que $y$ é a fração que resta do reagente, $t$ é o tempo, $T$ é a temperatura, por fim $k_o$ e $E$ são os parâmetros a serem estimados.



# Importação de pacotes

Importação de bibliotecas (pacotes) necessárias para execução do código.

* MT_PEU: biblioteca que contém as principais funcionalidades da ferramenta
* numpy: biblioteca para computação científica -> será necessária somente a função exp (exponencial)

In [34]:
from MT_PEU import EstimacaoNaoLinear
from numpy import exp

# Criação de modelo

O modelo, que é similar ao Exemplo_1, k0 e E são os parametros a serem estimados, diferindo apenas a equação do modelo. O mesmo é definido na forma de uma subrotina (def) do python e representa a equação (1):

In [35]:
def Modelo (param, x, args):

    tempo = x[:,0:1] # primeira coluna do vetor de dado
    T     = x[:,1:2] # segunda coluna do vetor de dados

    ko = param[0] # primeiro parâmetro
    E  = param[1] # segundo parâmetro

    y = exp(-tempo*exp(ko-E/T)) # cálculo de y - vetorizado

    return y

# Inicialização de classe

Neste exemplo apresentamos alumas possíveis entradas opcionais, como unidades, assim como a opção de renomear a pasta onde são gerados os aquivos com os resultados.
No exemplo a baixo, onde o nome da pasta foi alterado para 'Teste1'.


In [36]:
Estime = EstimacaoNaoLinear(Modelo, simbolos_x=[r't','T'], unidades_x=['s','K'],
                            simbolos_y=[r'y'], unidades_y=['adm'],
                            simbolos_param=['ko','E'], unidades_param=['adm','K'], projeto='Teste1')


# Inclusão de dados

Os dados experimentais da variável dependente (y) e das variáveis independentes (t e T)
são disponibilizados em Schwaab e Pinto (2007, p.326), e apresentados abaixo na forma de listas:


In [37]:
#Tempo
tempo = [120.0,60.0,60.0,120.0,120.0,60.0,60.0,30.0,15.0,60.0,
45.1,90.0,150.0,60.0,60.0,60.0,30.0,90.0,150.0,90.4,120.0,
60.0,60.0,60.0,60.0,60.0,60.0,30.0,45.1,30.0,30.0,45.0,15.0,30.0,90.0,25.0,
60.1,60.0,30.0,30.0,60.0]

#Temperatura
temperatura = [600.0,600.0,612.0,612.0,612.0,612.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,631.0,631.0,631.0,631.0,631.0,639.0,639.0,
639.0,639.0,639.0,639.0,639.0,639.0,639.0]

y = [0.9,0.949,0.886,0.785,0.791,0.890,0.787,0.877,0.938,
0.782,0.827,0.696,0.582,0.795,0.800,0.790,0.883,0.712,0.576,0.715,0.673,
0.802,0.802,0.804,0.794,0.804,0.799,0.764,0.688,0.717,0.802,0.695,0.808,
0.655,0.309,0.689,0.437,0.425,0.638,.659,0.449]

Como entrada obrigatória, a plataforma MT_PEU necessita da **incerteza dos dados experimentais** (ux1, ux2, uy1).
Neste exemplo, foram adotados o valor 1 para as incertezas.

In [38]:
uxtempo = [1]*41
uxtemperatura = [1]*41
uy = [1]*41

Inclusão de dados experimentais na estimação:
Inclui os dados experimentais nesse objeto *(setDados)*, onde a opção 0 é para a grandeza dependente,
e a opção 1 é para a grandeza independente.


In [39]:
Estime.setDados(0,(tempo,uxtempo),(temperatura,uxtemperatura))
Estime.setDados(1,(y,uy))

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

In [40]:
Estime.setConjunto()

# Otimização

Neste exemplo, o usuário tem a opção de escolha do algoritmo a ser utilizado na otimização. 
Disponiveis: 'Nelder-Mead', 'Powell', 'BFGS', 'L-BFGS-B', 'CG'. 
Caso opte por não escolher, será utilizado o algoritmo default: Nelder-Mead, com estimativa inicial em [0.03 , 20000.00].


In [41]:
#Estime.otimiza(estimativa_inicial= [0.03, 20000.000],algoritmo='Nelder-Mead')
Estime.SETparametro([3.900e+01, 2.764e+04])

# Incerteza


Neste exemplo é possível escolher o método útilizado para avaliar a incerteza 
Métodos disponíveis: 2InvHessiana, Geral, SensibilidadeModelo. 
Por definição o preenchimeto da região de verossimilhança é 'True', caso necessário esta opção pode ser alterada.


In [42]:
Estime.incertezaParametros(delta=1e-5,metodoIncerteza='2InvHessiana',preencherregiao=True)

# Predição de análise de resíduos

 No método predição, é feita a avaliação da grandeza dependente com base nos parametros fornecidos. 
 A covariância é avaliada, e consequentemente a eficiencia do modelo. 
 Em analise de residuos é possível vericar possíveis relações de dependencia e/ou tendencia entre as variaveis. 
 Testes estatisticos como o de homocedasticidade, chi quadrado, dentre outros são realizados
 nesta etapa. A analise de residuos é feita prioritariamente com os dados de validação.


In [43]:
Estime.predicao()
Estime.analiseResiduos()

# Gráficos e relatório

Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito.
dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.


In [44]:
etapas = ['otimizacao','grandezas-entrada', 'predicao','grandezas-calculadas','analiseResiduos', 'regiaoAbrangencia']
Estime.graficos(etapas)
Estime.relatorio()

/home/giufba-admin/Documentos/MT_PEU/MT_PEU.py:2084: UserWarning: Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.
  warn('Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.',UserWarning)


## Referências: 

SCHWAAB, M.M.;PINTO, J.C. Análise de Dados Experimentais I: Fundamentos da Estátistica e Estimação de Parâmetros. 
Rio de Janeiro: e-papers, 2007.

Avaliação de dados de medição — Guia para a expressão de incerteza de medição 
http://www.inmetro.gov.br/noticias/conteudo/iso_gum_versao_site.pdf 